In [43]:
import json


text="""\n{"Test plan": "\\ttc_mod "\n}\n"""

# text = text.replace("\r\n", "\\n").replace('\"', '\\"').replace("'", "\'")

json_data = json.loads(text)


In [37]:
import re
import json

def fix_invalid_json_escapes(json_str):
    # 有效的JSON转义序列
    valid_escapes = ['"', '\\', '/', 'b', 'f', 'n', 'r', 't', 'u']
    
    # 查找所有反斜杠后跟着的字符
    def replace_invalid_escape(match):
        escape_char = match.group(1)
        print(escape_char+"xx")
        # 如果是有效的转义序列，保留它
        if escape_char in valid_escapes:
            return '\\' + escape_char
        # 如果是无效的转义序列，在反斜杠前再加一个反斜杠使其成为字面量
        else:
            return '\\\\' + escape_char
    
    # 使用正则表达式查找和替换所有转义序列
    fixed_str = re.sub(r'\\([^"])', replace_invalid_escape, json_str)
    print(fixed_str)
    # 测试修复后的字符串是否可以被正确解析
    try:
        json.loads(fixed_str)
        return fixed_str
    except json.JSONDecodeError as e:

        print(f"修复后仍有错误: {e}")
        return None

# 测试示例
text="""{"Test plan": "\\ \\"111  "}"""

fixed_text = fix_invalid_json_escapes(text)
# print(fixed_text)

if fixed_text:
    parsed_json = json.loads(fixed_text)
    # print(parsed_json)

 xx
{"Test plan": "\\ \"111  "}


In [121]:
import re

def extract_components(text):
    thought_pattern = r'\*\*Thought\*\*: (.*?)\n'
    action_name_pattern = r'```(.*?)\n'
    action_params_pattern = r'```.*?\n(.*?)```'
    expected_info_pattern = r'\*\*Expected Information\*\*: (.*?)\n'
    
    pair_pattern = r'#### Thought-Action Pair (TA\d+)(.*?)(?=#### Thought-Action Pair TA\d+|\Z)'
    pairs = re.findall(pair_pattern, text, re.DOTALL)
    
    results = []
    
    for pair_id, pair_content in pairs:
        thought_match = re.search(thought_pattern, pair_content)
        action_name_match = re.search(action_name_pattern, pair_content)
        action_params_match = re.search(action_params_pattern, pair_content, re.DOTALL)
        expected_info_match = re.search(expected_info_pattern, pair_content)
        
        if thought_match and action_name_match and action_params_match and expected_info_match:
            try:
                action_params_json = json.loads(action_params_match.group(1).strip())
            except json.JSONDecodeError:
                action_params_json = {"error": "Invalid JSON"}
            
            pair_dict = {
                "id": pair_id.strip(),
                "thought": thought_match.group(1).strip(),
                "action_name": action_name_match.group(1).strip(),
                "action_parameters": action_params_json,
                "expected_information": expected_info_match.group(1).strip()
            }
            
            results.append(pair_dict)

# 示例文本
text = '''
### Exploration Step 2                                                                                  
                                                                                                                                                                 
#### Thought-Action Pair TA004                                                                                                                                   
- **Thought**: Since the `load_module` function has been modified, I need to understand how it interacts with other components in the system to ensure that all d
ependencies are correctly handled.                                                                                                                               
- **Action**: search_code_dependencies                                                                                                                           
- **Action Parameters**: {"entity_name": "load_module"}                                                                                                          
- **Expected Information**: This will help me identify which functions or classes call `load_module` and which functions it calls, allowing me to understand the 
broader impact of these changes.                                                                                                                                 
                                                                                                                                                                 
#### Thought-Action Pair TA005                                                                                                                                   
- **Thought**: The PR involves changes to the deck configuration, specifically the introduction of fixture definitions for modules. I need to understand how thes
e changes affect the overall deck configuration validation.                                                                                                      
- **Action**: search_class_in_project                                                                                                                            
- **Action Parameters**: {"class_name": "DeckConfiguration"}                                                                                                     
- **Expected Information**: This will provide details about the `DeckConfiguration` class, including its methods and properties, which are likely involved in val
idating the new fixture definitions.                                                                                                                             
                                                                                                                                                                 
#### Thought-Action Pair TA006                                                                                                                                   
- **Thought**: The PR includes updates to the `/deck_configuration` endpoint. I need to examine the changes to this endpoint to understand how it handles module 
fixtures.                                                                                                                                                        
- **Action**: view_code_changes                                                                                                                                  
- **Action Parameters**: {"file_path": "api/src/opentrons/calibration_storage/deck_configuration.py"}                                                            
- **Expected Information**: This will show me the exact changes made to the endpoint, including any new functionality or modifications to existing logic.        
                                                                                                                                                                 
#### Thought-Action Pair TA007                                                                                                                                   
- **Thought**: The PR modifies several files related to the protocol engine and state management. I need to understand how these changes affect the overall state
 management and validation of deck configurations.                                                                                                               
- **Action**: search_class_in_project                                                                                                                            
- **Action Parameters**: {"class_name": "State"}                                                                                                                 
- **Expected Information**: This will help me understand the `State` class, which is likely central to managing and validating deck configurations, including the
 new fixture definitions.                                                                                                                                        
                                                                                                                                                                 
#### Thought-Action Pair TA008                                                                                                                                   
- **Thought**: The PR includes changes to the `types.py` files in both the `calibration_storage` and `protocol_engine` directories. I need to examine these chang
es to understand how they impact the type definitions used throughout the system.                                                                                
- **Action**: view_code_changes                                                                                                                                  
- **Action Parameters**: {"file_path": "api/src/opentrons/calibration_storage/types.py"}
- **Expected Information**: This will show me the changes to the type definitions, which are crucial for ensuring consistency and correctness across the system.
'''

result = extract_components(text)
result


In [174]:
import os
import json
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib库

react_dir = 'result/Embedding'

react_list_dir = os.listdir(react_dir)

scores_file_list_dir = []
scores_list = []
for i in react_list_dir:
    scores_dir = os.path.join(react_dir, i, 'scores')
    if os.path.isdir(scores_dir):
        scores_list_dir = os.listdir(scores_dir)
        for score_file in scores_list_dir:
            if 'deepseek-chat' in score_file:
                score_file_path = os.path.join(scores_dir, score_file)
                # print(score_file_path)
                with open(score_file_path, 'r') as f: 
                    score = json.load(f)
                    scores_list.append(score)

In [87]:
import os
import json
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib库

react_dir = 'result/ReAct'

react_list_dir = os.listdir(react_dir)

scores_file_list_dir = []
for i in react_list_dir:
    scores_dir = os.path.join(react_dir, i, 'scores')
    if os.path.isdir(scores_dir):
        scores_list_dir = os.listdir(scores_dir)
        for score_file in scores_list_dir:
            score_file_path = os.path.join(scores_dir, score_file)
            if 'qwen-coder-14B' in score_file_path:
                pull_number = score_file_path.split('_')[-1].split('.')[0].strip()
                scores_file_list_dir.append(pull_number)

# print(scores_file_list_dir)
with open('/data/veteran/project/TestPlanAgent/data/PR/PR_URL_for_test.txt', 'r') as f:
    url_list = f.readlines()

pull_list = []
for url in url_list:
    pull_number = url.split('/')[-1].strip()
    pull_list.append({pull_number: url})

for p1 in pull_list:
    if list(p1.keys())[0] not in scores_file_list_dir:
        print(list(p1.values())[0].strip())
        

In [91]:
# 计算生成的test plan的得分情况
import os
import json
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib库

def cal(score_list, metric):
    print(f"metric is {metric}------------")
    average_score = sum(score_list) / len(score_list)

    # 计算中位数
    median_score = statistics.median(score_list)

    # 计算最大值
    max_score = max(score_list)

    # 计算最小值
    min_score = min(score_list)
    print(len(score_list))
    # 打印结果
    print(f"Average Score: {average_score}")
    print(f"Median Score: {median_score}")
    print(f"Max Score: {max_score}")
    print(f"Min Score: {min_score}")

react_dir = 'result/ReAct'

react_list_dir = os.listdir(react_dir)

scores_file_list_dir = []
scores_list = []
file_list = []
for i in react_list_dir:
    scores_dir = os.path.join(react_dir, i, 'scores')
    if os.path.isdir(scores_dir):
        scores_list_dir = os.listdir(scores_dir)
        for score_file in scores_list_dir:
            if 'qwen-coder-14B' in score_file:
                score_file_path = os.path.join(scores_dir, score_file)
                pull_number = score_file_path.split('_')[-1].split('.')[0].strip()
                if pull_number not in file_list:
                    file_list.append(pull_number)
                else:
                    print(pull_number)
                # print(score_file_path)
                with open(score_file_path, 'r') as f: 
                    score = json.load(f)
                    # if score['evaluation']['clarity']['score'] >= 5:
                    scores_list.append(score)

score_list = []
for score in scores_list:
    score_list.append(score['evaluation']['total_score'])
cal(score_list, "total_score")

score_list.clear()
for score in scores_list:
    score_list.append(score['evaluation']['accuracy']['score'])
cal(score_list, "accuracy")

score_list.clear()
for score in scores_list:
    score_list.append(score['evaluation']['completeness']['score'])
cal(score_list, "completeness")

score_list.clear()
for score in scores_list:
    score_list.append(score['evaluation']['clarity']['score'])
cal(score_list, "clarity")

metric is total_score------------
150
Average Score: 20.333333333333332
Median Score: 21.0
Max Score: 29
Min Score: 9
metric is accuracy------------
150
Average Score: 6.066666666666666
Median Score: 6.0
Max Score: 9
Min Score: 1
metric is completeness------------
150
Average Score: 6.24
Median Score: 6.0
Max Score: 10
Min Score: 1
metric is clarity------------
150
Average Score: 8.026666666666667
Median Score: 8.0
Max Score: 10
Min Score: 4


In [42]:
with open("/data/veteran/project/TestPlanAgent/data/PR/PR_URL_for_test.txt", 'r') as f:
    url_list = f.readlines()

pull_list = []

for url in url_list:
    pull_number = url.split('/')[-1].strip()
    pull_list.append(pull_number)

# print(len(pull_list))
# print(len(file_list))
for pull_number in file_list:
    if pull_list.count(pull_number) == 0:
        print(pull_number)
# 将两个list写入文件

# for pull_number in pull_list:
#     if pull_number not in file_list:
#         print(pull_number)

16024,


In [18]:
import os
import json
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib库

react_dir = 'result/ReAct'

react_list_dir = os.listdir(react_dir)

scores_file_list_dir = []
scores_list = []
for i in react_list_dir:
    scores_dir = os.path.join(react_dir, i, 'scores')
    if os.path.isdir(scores_dir):
        scores_list_dir = os.listdir(scores_dir)
        for score_file in scores_list_dir:
            if 'qwen-coder-14B' in score_file:
                score_file_path = os.path.join(scores_dir, score_file)
                # print(score_file_path)
                with open(score_file_path, 'r') as f: 
                    score = json.load(f)
                    if "evaluation" not in score:
                        scores_list.append(score_file_path)
scores_list




[]

In [21]:
import os
import json
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib库

react_dir = 'result/ReAct'

react_list_dir = os.listdir(react_dir)

scores_file_list_dir = []
scores_list = []
for i in react_list_dir:
    scores_dir = os.path.join(react_dir, i, 'scores')
    if os.path.isdir(scores_dir):
        scores_list_dir = os.listdir(scores_dir)
        for score_file in scores_list_dir:
            if 'qwen-coder-32B' in score_file:
                score_file_path = os.path.join(scores_dir, score_file)
                # print(score_file_path)
                with open(score_file_path, 'r') as f: 
                    score = json.load(f)
                    score['path'] = score_file_path
                    scores_list.append(score)

# for score in scores_list:
#     if score['evaluation']['clarity']['score'] < 5:
#         print(score['evaluation']['total_score'], score['path'], score['evaluation']['clarity']['score'])

with open('/data/veteran/project/TestPlanAgent/data/PR/PR_URL_for_test.txt', 'r') as f:
    full_url = f.readlines()

aim_list = []

for score in scores_list:
    if score['evaluation']['accuracy']['score'] < 5:
        for url in full_url:
            pull_number = score['path'].split('_')[-1].split('.')[0]
            if pull_number in url:
                aim_list.append(url)
                print(score['evaluation']['accuracy']['score'])
                break
aim_list
# with open('/data/veteran/project/TestPlanAgent/data/PR/PR_URL_for_test_resume_1.txt', 'w') as f:
#     f.writelines(aim_list)

    

In [197]:
import os
import json
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib库

react_dir = 'result/ReAct'

react_list_dir = os.listdir(react_dir)

scores_file_list_dir = []
scores_list = []
for i in react_list_dir:
    scores_dir = os.path.join(react_dir, i, "Test-Plan")
    if os.path.isdir(scores_dir):
        scores_list_dir = os.listdir(scores_dir)
        for score_file in scores_list_dir:
            flag = False
            if 'gpt-4o' in score_file:
                score_file_path = os.path.join(scores_dir, score_file)
                # print(score_file_path)
                with open(score_file_path, 'r') as f: 
                    lines = f.readlines()
                    # print(lines)
                    # break
                    for line in lines:
                        if "4. Test Cases" in line:
                            flag = True
                            break
                    if not flag:
                        scores_list.append(score_file_path)

print(len(scores_list))                   
for score in scores_list:
   print(score)

    

0


In [399]:
import os 
# 计算BLEU/ROUGE
result_dir = "./result/backup-with-git-diff-full/ReAct"
compare_pair = {}
# 获取ref test plan以及 candidate test plan
for repo in os.listdir(result_dir):
    repo_dir = os.path.join(result_dir, repo)
    if os.path.isdir(repo_dir):
        PR_content_dir = os.path.join(repo_dir, "PR-Content")
        if os.path.isdir(PR_content_dir):
            PR_content_list = os.listdir(PR_content_dir)
            for PR_content in PR_content_list:
                pull_number = PR_content.split("_")[0]
                PR_content_file_dir = os.path.join(PR_content_dir, PR_content)
                if os.path.isfile(PR_content_file_dir):
                    with open(PR_content_file_dir, 'r') as f:
                        PR_content_json = json.load(f)
                    compare_pair[pull_number] = []
                    compare_pair[pull_number].append(PR_content_json["Test_Plan"])
        Test_plan_dir = os.path.join(repo_dir, "Test-Plan")
        if os.path.isdir(Test_plan_dir):
            Test_plan_list = os.listdir(Test_plan_dir)
            for Test_plan in Test_plan_list:
                if "gpt-4o" in Test_plan:
                    PR_content_file_dir = os.path.join(Test_plan_dir, Test_plan)
                    pull_number = PR_content_file_dir.split("_")[-1].split(".")[0]
                    if os.path.isfile(PR_content_file_dir):
                        with open(PR_content_file_dir, 'r') as f:
                            PR_content_json = json.load(f)
                        compare_pair[pull_number].append(PR_content_json["react_info"][-1]["test_plan"])
# print(compare_pair['16418'][0])
for value in compare_pair.values():
    if len(value) != 2:
        print(value)
        print("error")
        break


In [400]:
import evaluate

bleu = evaluate.load('sacrebleu')
rouge = evaluate.load('rouge')

# For BLEU
# predictions = ["Going to play basketball this afternoon", "Going to play basketball afternoon"]
# references = [["Going to play basketball in the afternoon ?"], ["Going to play basketball in the afternoon ?"]]
references = [[value[0]] for value in compare_pair.values()]
predictions =[value[1] for value in compare_pair.values()]

bleu_results = bleu.compute(
    predictions=predictions, 
    references=references,
    # max_order=4,  # 只考虑1-gram和2-gram
    # smooth=True   # 启用平滑处理
)

# For ROUGE
rouge_results = rouge.compute(predictions=predictions, references=references)

print(f"BLEU: {bleu_results}")
print(f"ROUGE: {rouge_results}")

BLEU: {'score': 1.2435569065346208, 'counts': [3869, 834, 274, 105], 'totals': [44443, 44406, 44369, 44332], 'precisions': [8.705532929820219, 1.8781245777597622, 0.6175482882192522, 0.23684922854822701], 'bp': 1.0, 'sys_len': 44443, 'ref_len': 8309}
ROUGE: {'rouge1': 0.15005065305894255, 'rouge2': 0.03301212846981086, 'rougeL': 0.08387436333615349, 'rougeLsum': 0.14303829609377317}


In [324]:
import sacrebleu

# predictions = ["It's going to rain tomorrow."]
# references = [["It will rain tomorrow."]]

# 使用sacrebleu计算
bleu = sacrebleu.corpus_bleu(predictions, references, force=True, tokenize='none')

# 可通过调整输出分数得到paper中的计算方式
bleu

BLEU = 1.28 39.6/5.4/0.2/0.1 (BP = 1.000 ratio = 1.146 hyp_len = 636 ref_len = 555)

In [355]:
# 计算bertscore
from bert_score import score

references = [value[0] for value in compare_pair.values()]
predictions =[value[1] for value in compare_pair.values()]

# references = ["I want to go to lunch."]
# predictions = ["I'd like to go to lunch."]

P, R, F1 = score(predictions, references, lang="en", rescale_with_baseline=True)

print(f"System level F1 score: {F1.mean():.3f}")
print("Precision:", P)
print("Recall:", R)
print("F1 Score:", F1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


System level F1 score: -0.198
Precision: tensor([-0.3078, -0.2525, -0.2528, -0.2436, -0.1767, -0.3818, -0.2637,  0.0091,
        -0.1118, -0.0834, -0.0840, -0.2777, -0.3577, -0.1670, -0.2006, -0.0483,
        -0.0659, -0.2985, -0.1857, -0.2751, -0.1607, -0.1606, -0.2161, -0.1937,
        -0.2022, -0.1191, -0.1154, -0.0875, -0.1723, -0.1360, -0.2139, -0.2572,
        -0.5435, -0.2333, -0.1953, -0.5516, -0.5182])
Recall: tensor([-0.5200, -0.4521, -0.1914, -0.4054, -0.0300, -0.7674, -0.5075, -0.0966,
        -0.2609, -0.1439, -0.0098,  0.1636,  0.0916, -0.1436, -0.2203,  0.1602,
         0.0121, -0.3713, -0.0374, -0.4466, -0.0922, -0.1319, -0.0737, -0.2150,
        -0.1180, -0.0517, -0.0586, -0.1487, -0.1213, -0.0477,  0.0062,  0.0119,
        -0.3816, -0.1589, -0.1980, -0.2936, -0.3203])
F1 Score: tensor([-0.4141, -0.3522, -0.2203, -0.3238, -0.1027, -0.5806, -0.3866, -0.0426,
        -0.1856, -0.1120, -0.0455, -0.0653, -0.1417, -0.1534, -0.2085,  0.0553,
        -0.0256, -0.3330, -0.1109

In [342]:
print(f"System level F1 score: {F1.mean():.3f}")

System level F1 score: 0.798


In [227]:
import os 
# 计算BLEU
result_dir = "result/backup-without-git-diff/InOut"
generate_test_plan_path = "result/backup-without-git-diff/InOut/gpt-4o_claude-3-7-sonnet-20250219_result.json"
compare_pair = {}
# 获取ref test plan以及 candidate test plan
for repo in os.listdir(result_dir):
    repo_dir = os.path.join(result_dir, repo)
    if os.path.isdir(repo_dir):
        PR_content_dir = os.path.join(repo_dir, "PR-Content")
        if os.path.isdir(PR_content_dir):
            PR_content_list = os.listdir(PR_content_dir)
            for PR_content in PR_content_list:
                pull_number = PR_content.split("_")[0]
                PR_content_file_dir = os.path.join(PR_content_dir, PR_content)
                if os.path.isfile(PR_content_file_dir):
                    with open(PR_content_file_dir, 'r') as f:
                        PR_content_json = json.load(f)
                    compare_pair[pull_number] = []
                    compare_pair[pull_number].append(PR_content_json["Test_Plan"])

with open(generate_test_plan_path, 'r') as f:
    gen_test_plan = json.load(f)

for url in gen_test_plan:
    pull_number = url.split("/")[-1]
    if pull_number in compare_pair:
        compare_pair[pull_number].append(gen_test_plan[url]["test_plan"])
            

### Test Plan Details:
```
# Test Plan for PR: feat(commit-context): Allow committers from outside the org to be Suspect Committers

## 1. Purpose
The purpose of this test plan is to verify the functionality and correctness of allowing committers from outside the organization to be treated as Suspect Committers in the Sentry application. This involves validating changes made to handle both user and department committers, ensuring proper serialization and handling scenarios where both user and actor data might be null.

## 2. Scope
In Scope:
- Changes to the serialization logic in `event_file_committers.py`, particularly handling committers who are not within the Sentry organization.
- Modifications to the `Actor` and `GroupOwner` models to handle null scenarios and refine logic around users and teams.
- Updates to the `handle_auto_assignment` method in `projectownership.py`, ensuring correct assignments are only made when valid owners are resolved.
- Test cases related to new and modif

In [ ]:
# 统计工具的使用情况 适应另外一种格式的输出

import os 
# 计算BLEU
result_dir = "./result/ReAct"

reacts = []
# 获取ref test plan以及 candidate test plan
for repo in os.listdir(result_dir):
    repo_dir = os.path.join(result_dir, repo)
    if os.path.isdir(repo_dir):
        Test_plan_dir = os.path.join(repo_dir, "Test-Plan")
        if os.path.isdir(Test_plan_dir):
            Test_plan_list = os.listdir(Test_plan_dir)
            for Test_plan in Test_plan_list:
                if "gpt-3.5-turbo" in Test_plan:
                    PR_content_file_dir = os.path.join(Test_plan_dir, Test_plan)
                    pull_number = PR_content_file_dir.split("_")[-1].split(".")[0]
                    if os.path.isfile(PR_content_file_dir):
                        with open(PR_content_file_dir, 'r') as f:
                            PR_content_json = json.load(f)
                        reacts.extend(PR_content_json["react_info"][:-1])
        scores_dir = os.path.join(repo_dir, "scores")
        if os.path.isdir(scores_dir):
            scores_list_dir = os.listdir(scores_dir)
            for score_file in scores_list_dir:
                if 'gpt-3.5-turbo' in score_file:
                    score_file_path = os.path.join(scores_dir, score_file)
                    # print(score_file_path)
                    pull_number = score_file_path.split("_")[-1].split(".")[0]
                    with open(score_file_path, 'r') as f: 
                        score = json.load(f)
                        reacts.append(score)


{'thought': 'I need to look at the implementation details of the `ReplayView` function, as the PR mentions changes related to how the replay video resizing happens when the browser window size changes. Understanding this implementation will enable me to generate comprehensive test cases tailored to the introduced changes.',
 'action': 'search_function_in_project',
 'action_param': {'function_name': 'ReplayView'},
 'observation': 'cat not find the entity (class or function) in the project'}

In [241]:
# 统计工具的使用情况
tool_list = {}
for react in reacts:
    tool = react["action"]
    if tool not in tool_list:
        tool_list[tool] = {}
        tool_list[tool]["number"]= 1
        tool_list[tool]["observation"] = []
        tool_list[tool]["observation"].append(react["observation"])
    else:
        tool_list[tool]["number"] += 1
        tool_list[tool]["observation"].append(react["observation"])
print(tool_list)

{'search_function_in_project': {'number': 56, 'observation': ['cat not find the entity (class or function) in the project', 'cat not find the entity (class or function) in the project', '{"error": "Entity \'get\' exists but has invalid structure"}', '{"detail_of_entity": {"category": "function", "info": "    def save(self, commit=True):\\n        user = super().save(commit=False)\\n        user.set_password(self.cleaned_data[\\"password\\"])\\n        if commit:\\n            user.save()\\n            if self.cleaned_data.get(\\"subscribe\\"):\\n                newsletter.backend.create_or_update_subscriptions(\\n                    user, list_ids=newsletter.backend.get_default_list_ids()\\n                )\\n            if self.cleaned_data.get(\\"timezone\\"):\\n                UserOption.objects.create(\\n                    user=user, key=\\"timezone\\", value=self.cleaned_data.get(\\"timezone\\")\\n                )\\n        return user", "fname": "/data/veteran/project/TestPlan

In [242]:

with open("tool_list.json", 'w') as f:
    json.dump(tool_list, f, indent=4)

In [366]:
# 统计工具使用后得分情况
import os 
result_dir = "./result/ReAct"

llm_model = "qwen-coder-32B"

reacts_and_scores = {}
# 获取ref test plan以及 candidate test plan
for repo in os.listdir(result_dir):
    repo_dir = os.path.join(result_dir, repo)
    if os.path.isdir(repo_dir):
        Test_plan_dir = os.path.join(repo_dir, "Test-Plan")
        if os.path.isdir(Test_plan_dir):
            Test_plan_list = os.listdir(Test_plan_dir)
            for Test_plan in Test_plan_list:
                if llm_model in Test_plan:
                    PR_content_file_dir = os.path.join(Test_plan_dir, Test_plan)
                    pull_number = PR_content_file_dir.split("_")[-1].split(".")[0]
                    reacts_and_scores[pull_number] = {}
                    if os.path.isfile(PR_content_file_dir):
                        with open(PR_content_file_dir, 'r') as f:
                            PR_content_json = json.load(f)
                        reacts_and_scores[pull_number]["react_info"] = PR_content_json["react_info"][:-1]
                    
        scores_dir = os.path.join(repo_dir, "scores")
        if os.path.isdir(scores_dir):
            scores_list_dir = os.listdir(scores_dir)
            for score_file in scores_list_dir:
                if llm_model in score_file:
                    score_file_path = os.path.join(scores_dir, score_file)
                    # print(score_file_path)
                    pull_number = score_file_path.split("_")[-1].split(".")[0]
                    reacts_and_scores[pull_number]["scores"] = []
                    with open(score_file_path, 'r') as f: 
                        score = json.load(f)
                        score["type"] = "ReAct"
                        reacts_and_scores[pull_number]["scores"].append(score)
                    # inout_score_file_path = score_file_path.replace("ReAct", "backup-without-git-diff/InOut")
                    inout_score_file_path = score_file_path.replace("ReAct", "InOut")
                    with open(inout_score_file_path, 'r') as f:
                        score = json.load(f)
                        score["type"] = "InOut"
                        reacts_and_scores[pull_number]["scores"].append(score)
# print(reacts_and_scores)
with open(f"{llm_model}_reacts_and_scores_summary_diff.json", 'w') as f:
    json.dump(reacts_and_scores, f, indent=4)
